In [13]:
from os import listdir
from os.path import isfile, join
import os
import numpy as np
from amrafile import amrafile as af
from amracommon.analysis.registration import normalized_cross_correlation

testTargets =  ["0013A","0013B","0013C","0013D","0013E","0013F","00130","00131",
     "00132","00133","00134","00135","00136","00137","0014A","0014B",
     "0014D","0014F","00140","00141","00142","00143","00144","00145",
     "00146","00147","0015A","0015B","0015C","0015D","0015E","0015F",
     "00152","00154","00155","00156","00157","00158","00159"]

#testTargets = ["00150"]
reduced_size = [7,10,10]

error = []

for target in testTargets:

    signal = af.parse('/moria/data/DB/0030/'+target+'/wholebody_normalized_water_1_'+target+'.amra')
    prototypePath = '/media/hannes/localDrive/DB/0030/'+target+'/prototypes'
    
    prototypes = [join(prototypePath,f) for f in listdir(prototypePath)  if isfile(join(prototypePath, f))]
    
    nbr_of_prototypes = len(prototypes)
    
    targetPoi = signal.get_poi('T9')
    voxelSize = signal.voxel_size()
    
    prototypeSignals = []
    for prototype in prototypes:
        prototypeSignals.append(af.parse(prototype))
        
    prototypesT9 = []
    for sig in prototypeSignals:
        prototypesT9.append(sig.get_poi('T9'))
    
    prototypesLF = []
    for sig in prototypeSignals:
        prototypesLF.append(sig.get_poi('LeftFemur'))
        
    ncc = []
    
    target = signal.data

    for ind, poi in enumerate(prototypesT9):

        z_lower = poi[0]-reduced_size[0]
        z_upper = poi[0]+reduced_size[0]+1
        y_lower = poi[1]-reduced_size[1]
        y_upper = poi[1]+reduced_size[1]+1
        x_lower = poi[2]-reduced_size[2]
        x_upper = poi[2]+reduced_size[2]+1

        prototype = prototypeSignals[ind].data

        ''' Extract reduced space from prototype and target'''
        reduced_prototype = prototype[z_lower:z_upper, y_lower:y_upper, x_lower:x_upper]
        reduced_target = signal.data[z_lower:z_upper, y_lower:y_upper, x_lower:x_upper]

        ''' Calculate ncc and store in lists'''
        ncc.append(normalized_cross_correlation(reduced_prototype, reduced_target))
        #prototype_mean = np.tile(np.mean(reduced_prototype),reduced_prototype.shape)
        #target_mean = np.tile(np.mean(reduced_target),reduced_target.shape)
        #prototype_std = np.std(reduced_prototype)
        #target_std = np.std(reduced_target)
        #ncc.append(np.sum(((reduced_prototype-prototype_mean)/prototype_std-(reduced_target-target_mean)/target_std)**2))
            
    poi_index = ncc.index(max(ncc))
    
    ''' Repmat the target poi to compare distances'''
    rep_target_poi = np.tile(targetPoi, (nbr_of_prototypes,1))
    rep_target_size = np.tile(voxelSize, (nbr_of_prototypes,1))

    ''' Diff between target poi ground truth and every deformed prototype poi in mm'''
    poi_diff = list(prototypesT9 - rep_target_poi)

    sorted_diff = np.array([(x,y) for (y,x) in sorted(zip(ncc,poi_diff))])

    ncc_diff = poi_diff[poi_index][0]
    error.append(ncc_diff)

    ''' Differences sorted in ascending order'''
    #sorted_diff = sorted(poi_diff)

    ''' Best poi to choose from prototypes '''
    #prototype_poi_index = poi_diff.index(min(poi_diff))
print(ncc_diff)    

4


In [17]:
error

[0,
 1,
 5,
 6,
 -8,
 3,
 12,
 11,
 1,
 0,
 -1,
 1,
 -2,
 -5,
 -4,
 -9,
 15,
 9,
 0,
 -2,
 0,
 -6,
 -2,
 10,
 -6,
 2,
 1,
 9,
 2,
 -6,
 -5,
 -2,
 7,
 8,
 10,
 1,
 2,
 4,
 4]